## Vector Search on Text: Azure Cognitive Search via Python SDK

### Prerequisites
  
- Generate embeddings - [generate_embeddings.ipynb](../../common/generate_embeddings.ipynb) 
- Create ACS Index and ingest embeddings - [cognitive_search_sdk_ingestion.ipynb](./cognitive_search_sdk_ingestion.ipynb)

### Set environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

acs_key  = os.getenv("COGNITIVE_SEARCH_KEY")
if acs_key is None or acs_key == "":
    print("COGNITIVE_SEARCH_KEY environment variable not set.")
    exit()

aoai_key  = os.getenv("AZURE_OPENAI_KEY")
if aoai_key is None or aoai_key == "":
    print("AZURE_OPENAI_KEY environment variable not set.")
    exit()
    
index_name_text = 'text-sample'
index_name_doc = 'doc-sample'
index_name_image = 'image-sample'
acs_endpoint = 'https://cogsearch02.search.windows.net'
acs_index_definition = 'index_definition/index_definition_text.json'
acs_api_version = '2023-07-01-Preview'
aoai_endpoint = 'https://azure-openai-dnai.openai.azure.com'
aoai_api_version = '2023-08-01-preview'
aoai_embedding_deployed_model = 'embedding-ada'

### Helper methods

In [ ]:
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchField,
    SearchableField,
    SearchFieldDataType,
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
    SemanticSettings,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SearchIndex
)

def vectorize_text_com_vision(com_vision_endpoint,com_vision_key,query):
    vectorize_text_url = f"{com_vision_endpoint}/computervision/retrieval:vectorizeText"  
    params = {  
        "api-version": "2023-02-01-preview"  
    } 
    headers = {  
        "Content-Type": "application/json",  
        "Ocp-Apim-Subscription-Key": com_vision_key  
    }  
    data = {
        'text':query
    }

    response = requests.post(vectorize_text_url, params=params, headers=headers, json=data)
    query_vector = response.json()["vector"]

    return query_vector

def show_image(image_folder, image):
    image_path = os.path.join(image_folder, image)
    plt.imshow(Image.open(image_path))
    plt.axis('off')
    plt.show()

### Perform a vector similarity search

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from azure.search.documents.models import Vector  

query = 'tools for software development'

query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)

search_client = SearchClient(acs_endpoint, index_name_text, AzureKeyCredential(acs_key))
vector = Vector(value=query_vector, k=3, fields="content_vector")  

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["title", "content"]  
)   

for result in results:
    print(result['title'])
    print("\n")

### Perform a vector similarity search on doc

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from azure.search.documents.models import Vector  

query = 'when are performance review announced?'
query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)

search_client = SearchClient(acs_endpoint, index_name_doc, AzureKeyCredential(acs_key))
vector = Vector(value=query_vector, k=3, fields="chunk_content_vector")  

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["chunk_content"]  
)   

for result in results:
    print(result['chunk_content'])
    print("\n")

### Perform a vector similarity search for image

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from azure.search.documents.models import Vector  

query = 'flower with hand'

query_vector = vectorize_text_com_vision(com_vision_endpoint,com_vision_key,query)

search_client = SearchClient(acs_endpoint, index_name_image, AzureKeyCredential(acs_key))
vector = Vector(value=query_vector, k=1, fields="image_vector")  

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["image"]  
)   

for result in results:
    show_image(image_folder, result['image'])
    print("\n")